In [2]:
from common import *

In [3]:
from numba import jit

# legit alg

**Big note: passing different length tuples to numba causes multiple compilations for the function!**

In [4]:
from heapq import *

try:
    del _alg_evaluate_homogeneous
    del _alg_evaluate_with_intercept
except NameError:
    pass

@jit(nopython=True)
def _lr_add_intercept(x):
    return np.concatenate((np.ones((x.shape[0], 1)), x), axis=1)

@jit(nopython=True)
def _lr_fit(x, y, w):
    wsqrt = w ** 0.5
    thing = np.linalg.lstsq(np.expand_dims(wsqrt, 1) * x, wsqrt * y)
    return thing[0]

@jit(nopython=True)
def _lr_predict(x, beta):
    return x @ beta.astype(x.dtype)

@jit(nopython=True)
def _lr_rss(y, y_, w):
    res = (y_ - y) ** 2 * w
    res.sort()
    return res.sum()

@jit(nopython=True)
def _alg_evaluate_homogeneous(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep, m, k, subk):
    cvs = np.empty(m, dtype=ycv.dtype) + np.nan
    eps = np.empty(m, dtype=yep.dtype) + np.nan
    model = np.empty((m, subk.shape[0]), dtype=xtr.dtype) + np.nan
    for g in range(m):
        xtrg, ytrg, wtrg, xcvg, ycvg, wcvg, xepg, yepg, wepg = (
            xtr[:, g::m][:, subk], ytr[:, g], wtr[:, g],
            xcv[:, g::m][:, subk], ycv[:, g], wcv[:, g],
            xep[:, g::m][:, subk], yep[:, g], wep[:, g])
        beta = _lr_fit(xtrg, ytrg, wtrg)
        model[g] = beta
        ycvg_ = _lr_predict(xcvg, beta)
        cvs[g] = _lr_rss(ycvg, ycvg_, wcvg)
        yepg_ = _lr_predict(xepg, beta)
        eps[g] = _lr_rss(yepg, yepg_, wepg)
    cvs.sort()
    eps.sort()
    return (cvs.sum(), eps.sum()), model

@jit(nopython=True)
def _alg_evaluate_with_intercept(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep, m, k, subk):
    cvs = 0.0
    eps = 0.0
    for g in range(m):
        xtrg, ytrg, wtrg, xcvg, ycvg, wcvg, xepg, yepg, wepg = (
            xtr[:, g::m][:, subk], ytr[:, g], wtr[:, g],
            xcv[:, g::m][:, subk], ycv[:, g], wcv[:, g],
            xep[:, g::m][:, subk], yep[:, g], wep[:, g])
        beta = _lr_fit(_lr_add_intercept(xtrg), ytrg, wtrg)
        ycvg_ = _lr_predict(_lr_add_intercept(xcvg), beta)
        cvs += _lr_rss(ycvg, ycvg_, wcvg)
        yepg_ = _lr_predict(_lr_add_intercept(xepg), beta)
        eps += _lr_rss(yepg, yepg_, wepg)
    return cvs, eps

def alg(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep, intercept=True):
    '''
    xtr.shape = (ntr, (m, k))
    ytr.shape = wtr.shape = (ntr, m)
    similar for cv, ep
    should not have NaNs (cells corresponding to where w==0 are multiplied by 0 anyway)
    no intercept term is fit on the x side, so make sure for every feature in x, 0 represents no information
    '''
    _alg_evaluate = [_alg_evaluate_homogeneous, None][intercept]
    k = xtr.columns.get_level_values(0).nunique() # features
    m = xtr.columns.get_level_values(1).nunique() # stocks/groups
    columns = xtr.columns
    features = set(range(k))
    baseline = (sum(_lr_rss(y, np.zeros_like(y), w) for y, w in zip(ycv.values, wcv.values)),
                sum(_lr_rss(y, np.zeros_like(y), w) for y, w in zip(yep.values, wep.values)))
    # the mean model uses the ytr and ycv, similar to as if yep was the leaderboard data and ytr and ycv were given
    # some stocks may have total 0 weight, so fillna for those columns
    meandata = (((ytr * wtr).sum() + (ycv * wcv).sum()) / (wtr.sum() + wcv.sum())).fillna(0)
    xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep = (
        xtr.values, (ytr - meandata).values, wtr.values,
        xcv.values, (ycv - meandata).values, wcv.values,
        xep.values, (yep - meandata).values, wep.values)
    meandatascore = (sum(_lr_rss(y, np.zeros_like(y), w) for y, w in zip(ycv, wcv)),
                 sum(_lr_rss(y, np.zeros_like(y), w) for y, w in zip(yep, wep)))
    scores = {None: baseline, (): meandatascore}
    models = {None: 0, (): meandata}
    heap = [(meanmodel[0], ())]
    while True:
        minscore0, minfeats = heappop(heap)
        print(minscore0 / baseline[0], scores[minfeats][1] / baseline[1], len(minfeats))
        print('(' + ', '.join((str(columns[m * f][0]) for f in minfeats) if minfeats else ()) + ')' + str(minfeats))
        print()
        print('[{}]'.format(len(features - set(minfeats))), end='')
        for ii, f in enumerate(features - set(minfeats)):
            print('.' if ii % 10 else ii, end='')
            #print('trying', f)
            newfeats = tuple(sorted(minfeats + (f,)))
            if newfeats not in scores:
                newscore, model = _alg_evaluate(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep,
                                                 m, k, np.array(newfeats, dtype=np.int64))
                scores[newfeats] = newscore
                models[newfeats] = model
                heappush(heap, (newscore[0], newfeats))
                #print('newguy', (newscore, newfeats))
            #print()
        print()
        if not heap or minscore0 <= heap[0][0]:
            break
    return scores, models

In [34]:
if __name__ == '__main__':
    Xm = np.array([
        [1, 1, 0, 0, 0, 1],
        [1, 1, 1, 2, 1.1, 0],
        [1, 1, 0, 4, 2, 1],
    ], dtype=np.float64)
    Y = np.array([
        [0, 2, 3],
        [1, 3, 4],
        [2, 4, 5],
    ], np.float64)
    W = np.array([
        [1, 1, 1],
        [3, 1, 1],
        [0, 1, 1],
    ], dtype=np.float64)
    score, model = _alg_evaluate_homogeneous(Xm, Y, W, Xm, Y, W, Xm, Y, W, 3, 2, np.array([1]))

### old implementations

#### high level pseudocode

In [ ]:
def pseudo_alg():
    lastminguy = None
    init(scores)
    init(heap)
    while True:
        minscore, minguy = heap.pop()[1]
        for f in features - minguy:
            newguy = minguy + {f}
            if newguy not in scores:
                score = eval(newguy)
                scores[newguy] = score
                heap += (score, newguy)
        if minscore <= heap[0][0]:
            break

#### naive no numba implementation

In [222]:
from heapq import *

def naive_alg(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep):
    groups = xtr.columns.get_level_values(0).unique()
    features = set(xtr.columns.get_level_values(1))
    m, k = len(groups), len(features)
    #
    #
    #@jit(nopython=True)
    def evaluate(feats):
        cvs = []
        eps = []
        for g in groups:
            beta = lr_fit(xtr.loc[:, (g, feats)].values, ytr.loc[:, g].values, wtr.loc[:, g].values)
            y_ = lr_predict(xcv.loc[:, (g, feats)].values, beta)
            cvs += [lr_rss(ycv.loc[:, g].values, y_, wcv.loc[:, g].values)]
            y_ = lr_predict(xep.loc[:, (g, feats)].values, beta)
            eps += [lr_rss(yep.loc[:, g], y_, wep.loc[:, g].values)]
        return sum(cvs), sum(eps)
    #
    baseline, baselinetest = evaluate(())
    scores, scorestest = {(): baseline}, {(): baselinetest}
    heap = [(baseline, ())]
    #
    while True:
        minscore, minfeats = heappop(heap)
        print(minscore / baseline, scorestest[minfeats] / baselinetest, len(minfeats))
        print(minfeats)
        print()
        for f in features - set(minfeats):
            #print('trying', f)
            newfeats = tuple(sorted(minfeats + (f,)))
            if newfeats not in scores:
                newscore, newscoretest = evaluate(newfeats)
                scores[newfeats], scorestest[newfeats] = newscore, newscoretest
                heappush(heap, (newscore, newfeats))
                #print('newguy', (newscore, newfeats))
            #print()
        if not heap or minscore <= heap[0][0]:
            break
    return

#### all numba failed implementation

In [18]:
@jit(nopython=True)
def _alg_evaluate(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep, m, k, subk):
    cvs = 0.0
    eps = 0.0
    for g in range(m):
        xtrg, ytrg, wtrg, xcvg, ycvg, wcvg, xepg, yepg, wepg = (
            xtr[:, g*k:(g+1)*k], ytr[:, g*k:(g+1)*k], wtr[:, g*k:(g+1)*k],
            xcv[:, g*k:(g+1)*k], ycv[:, g*k:(g+1)*k], wcv[:, g*k:(g+1)*k],
            xep[:, g*k:(g+1)*k], yep[:, g*k:(g+1)*k], wep[:, g*k:(g+1)*k])
        beta = _lr_fit(xtrg, ytrg, wtrg)
        ycvg_ = _lr_predict(xcvg, ycvg, wcvg)
        cvs += _lr_rss(ycvg, ycvg_, wcvg)
        yepg_ = _lr_predict(xepg, yepg, wepg)
        eps += _lr_rss(yepg, yepg_, wepg)
    return cvs, eps

@jit(nopython=True)
def _alg_help_featsin(scored, feats):
    pass

@jit(nopython=True)
def _alg_help_featsadd(scored, feats):
    pass

@jit(nopython=True)
def _alg_main(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep, maxnumfeats, m, k):
    baseline, baselinetest = _alg_evaluate(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep, m, k, ())
    scored = np.zeros((k, 1), np.bool)
    #scoreslist = []
    heap = {(baseline, ())}
    minscore = np.inf
    while True:
        newminscore, newminfeats = min(heap)
        heap.remove((newminscore, newminfeats))
        if minscore <= newminscore:
            break
        minscore, minfeats = newminscore, newminfeats
        if len(minfeats) >= maxnumfeats:
            break
        minfeatsset = set(minfeats)
        for f in [f for f in range(k) if f not in minfeatsset]:
            newfeats = tuple(sorted(minfeats + (f,)))
            if newfeats not in scored:
                newscore, newscoretest = _alg_evaluate(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep, m, k, newfeats)
                heap.add((newscore, newfeats))
                scored.add(newfeats)
                #scoreslist.append((newscore, newscoretest, newfeats))
        if not heap:
            break
    return scores[minfeats], scorestest[minfeats], minfeats

def alg_numba(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep):
    groups = xtr.columns.get_level_values(0).unique()
    features = set(xtr.columns.get_level_values(1))
    n, m, k = len(xtr), len(groups), len(features)
    cvs, eps, feats = _alg_main(
        xtr.values, ytr.values, wtr.values, xcv.values, ycv.values, wcv.values, xep.values, yep.values, wep.values,
        40, m, k)
    return cvs, eps, features[feats]

In [19]:
if __name__ == '__main__':
    Xtest = Xuse
    Ytest = Ya
    Wtest = Wa
    alg_numba(Xtest[tr], Ytest[tr], Wtest[tr], Xtest[cv], Ytest[cv], Wtest[cv], Xtest[ep], Ytest[ep], Wtest[ep])

TypingError: Failed at nopython (nopython frontend)
Untyped global name 'tuple': cannot determine Numba type of <class 'type'>
File "<ipython-input-18-d2daf27047d8>", line 33

In [98]:
if __name__ == '__main__':
    @jit(nopython=True)
    def tee(m, k):
        ll = np.zeros((m, k))
        for i in range(m):
            arr = np.random.randn(k)
            ll[i] = arr
        return ll
    tee(4, 3)

array([[-0.58976169, -0.43637558, -0.02656932],
       [-0.00891674, -1.3048404 ,  1.07547882],
       [ 1.02775793, -2.9534105 ,  0.63427395],
       [-0.80154266,  0.79637473, -0.25712827]])